# 作为服务供中型团队使用

中型团队指那种使用人数高于10人且规模但低于100人的基本固定的团队.这种团队一般是算法研发方面独角兽初创公司或者大公司的特定方向研发团队.通常会拥有不止一台工作站.我们以一台8显卡的工作站为例,假设一个工作站最多可以让20个人使用,那么也就只需要5个这样的工作站就可以支撑一个100人的研发团队.让他们在统一的jupyterhub平台进行研发既可以避免相互干扰弄坏环境,又可以统一工具避免不必要的沟通成本.

这种场景下我们可以沿用小型团队的资源,只是改为docker swarm集群化部署.

部署的例子我放在[项目]()下

## swarm环境下部署jupyterhub

下面是hub和proxy在100~150人负载时的资源占用情况:


Component | CPU (mean/peak) | Memory (mean/peak)
---|---|---
Hub| 4% / 13%| (230 MB / 260 MB)
Proxy| 6% / 13%| (47 MB / 65 MB)

可以看出单个hub和proxy实例就足以支撑这个规模的负载.

我们需要将hub服务部署在swarm的manager节点上,通常manager节点有条件的话应该独立于高算力机器,仅用于管理集群和提供接口.而那些高算力机器则应该作为worker挂载在swarm集群中.当然没条件那就凑活部署了.

对于swarm方式部署,更加推荐将hub和[proxy](https://hub.docker.com/r/jupyterhub/configurable-http-proxy)拆开部署.需要注意无论是hub还是proxy,都只能部署一个实例.不过不用担心,下面是

在集群环境下我们需要注意应当尽量将


